## 简答题
1. 如果已经在完全相同的训练集上训练了5个不同的模型，并且它们都达到了95%的准确率，是否还有机会获得更好的效果？如果可以，该怎么做？ 如果不行，为什么？
2. 硬投票分类器和软投票分类器有什么区别？
3. 是否可以通过在多个服务器上并行来加速bagging集成的训练？pasting集成呢？提升集成呢？随机森林或堆叠集成呢？
4. 包外评估的好处是什么？
5. 是什么让极端随机树比一般随机森林更加随机？这部分增加的随机性有什么用？极端随机树比一般随机森林快还是慢？
6. 如果AdaBoost集成对训练数据欠拟合，应该调整哪些超参数？怎么调整？
7. 如果梯度提升集成对训练集过拟合，应该提升还是降低学习率

## 编程题
1. 加载MNIST数据集（**8_sklearn做分类** 里有介绍），将其分为一个训练集、一个验证集和一个测试集（例如使用40000个实例训练，10000个实例验证，最后20000个实例测试）。然后训练多个分类器，比如一个随机森林分类器、一个极端随机树分类器和一个SVM。接下来，尝试使用软投票法或者硬投票法将它们组合成一个集成，这个集成在验证集上的表现要胜过它们各自单独的表现。成功找到集成后，在测试集上测试。与单个的分类器相比，它的性能要好多少？


2. 运行上一题中的单个分类器，用验证集进行预测，然后用预测结果创建一个新的训练集：新训练集中的每个实例都是一个向量，这个向量包含所有分类器对于一个图像的一组预测，目标值是图像的类。恭喜，你成功训练了一个混合器，结合第一层的分类器，它们一起构成了一个堆叠集成。现在在测试集上评估这个集成。对于测试集中的每个图像，使用所有的分类器进行预测，然后将预测结果提供给混合器，得到集成的预测。与前面训练的投票分类器相比，这个集成的结果如何？现在再次尝试使用StackingClassifier。你得到了更好的性能吗？如果是这样，为什么？

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False, parser="auto")

In [2]:
mnist.data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [3]:
X_train, X_verify,X_test = mnist.data[:40000] , mnist.data[40000:50000] , mnist.data[50000:]
y_train, y_verify,y_test = mnist.target[:40000] , mnist.target[40000:50000] , mnist.target[50000:]

In [4]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [5]:
rf = RandomForestClassifier(max_depth=5, n_estimators=100,max_leaf_nodes=100,random_state=42)      # 0.8548
# rf = RandomForestClassifier(random_state=42)            # 0.9607
# rf.fit(X_train,y_train)
# accuracy_score(y_verify,rf.predict(X_verify))


In [6]:
# etc = ExtraTreesClassifier(n_estimators=50,max_leaf_nodes=100,random_state=42)      # 0.8913
etc = ExtraTreesClassifier(n_estimators=20,max_leaf_nodes=100,min_samples_leaf=10,random_state=42)      # 0.8797
# etc = ExtraTreesClassifier(random_state=42)           # 0.9634
# etc.fit(X_train,y_train)
# accuracy_score(y_verify,etc.predict(X_verify))

In [7]:
sgd = SGDClassifier(loss = 'log_loss',random_state=42)
# sgd.fit(X_train,y_train)
# accuracy_score(y_verify,sgd.predict(X_verify))          # 0.8696

In [8]:
from sklearn.ensemble import VotingClassifier
voting_hard = VotingClassifier(estimators=[('rf', rf),('etc' , etc),('sgd',sgd)], voting='hard')
voting_soft = VotingClassifier(estimators=[('rf', rf),('etc' , etc),], voting='soft')

In [9]:
voting_hard.fit(X_train, y_train)
voting_soft.fit(X_train, y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=5,
                                                     max_leaf_nodes=100,
                                                     random_state=42)),
                             ('etc',
                              ExtraTreesClassifier(max_leaf_nodes=100,
                                                   min_samples_leaf=10,
                                                   n_estimators=20,
                                                   random_state=42))],
                 voting='soft')

In [10]:
accuracy_score(y_verify, voting_hard.predict(X_verify))

0.8889

In [11]:
accuracy_score(y_verify, voting_soft.predict(X_verify))

0.8807

In [12]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     # max_depth=5, n_estimators=100,max_leaf_nodes=100
#     "rf__max_depth": [5,6,7,],
#     "rf__n_estimators": [20,40,60,],
#     "rf__max_leaf_nodes": [100,150],
#     "etc__max_depth": [5,6,7],
#     "etc__n_estimators": [20,40,60,],
#     "etc__max_leaf_nodes": [100,150],
# }
#
# grad_search = GridSearchCV(estimator=voting_hard, param_grid=param_grid , cv=3, n_jobs=-1)


In [13]:
# grad_search.fit(X_train, y_train)

In [26]:
import numpy as np
X_train_all = np.concatenate((X_train, X_verify))
y_train_all = np.concatenate((y_train, y_verify))
X_train_all.shape, y_train_all.shape

((50000, 784), (50000,))

In [28]:
from sklearn.ensemble import StackingClassifier

stackclassifier = StackingClassifier(estimators=[('rf',rf),('etc',etc),],final_estimator=RandomForestClassifier(random_state=42),cv=5)
stackclassifier.fit(X_train_all, y_train_all)

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(max_depth=5,
                                                       max_leaf_nodes=100,
                                                       random_state=42)),
                               ('etc',
                                ExtraTreesClassifier(max_leaf_nodes=100,
                                                     min_samples_leaf=10,
                                                     n_estimators=20,
                                                     random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=42))

In [29]:
accuracy_score(y_verify, stackclassifier.predict(X_verify))

0.921

In [30]:
stackclassifier_hard = StackingClassifier(estimators=[('rf',rf),('etc',etc),('sgd',sgd)],final_estimator=RandomForestClassifier(random_state=42),cv=5,stack_method='predict')
stackclassifier_hard.fit(X_train_all, y_train_all)

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(max_depth=5,
                                                       max_leaf_nodes=100,
                                                       random_state=42)),
                               ('etc',
                                ExtraTreesClassifier(max_leaf_nodes=100,
                                                     min_samples_leaf=10,
                                                     n_estimators=20,
                                                     random_state=42)),
                               ('sgd',
                                SGDClassifier(loss='log_loss',
                                              random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=42),
                   stack_method='predict')

In [31]:
accuracy_score(y_verify, stackclassifier_hard.predict(X_verify))

0.9088

In [32]:
# voting_hard.score(X_test,y_test)                # 0.903
# voting_soft.score(X_test,y_test)                # 0.89295
# stackclassifier.score(X_test,y_test)            # 0.9224
# stackclassifier_hard.score(X_test,y_test)       # 0.9056

In [33]:
voting_hard.score(X_test,y_test) ,voting_soft.score(X_test,y_test) ,stackclassifier.score(X_test,y_test)  ,stackclassifier_hard.score(X_test,y_test)

(0.903, 0.89295, 0.9205, 0.91015)